In [69]:
import pandas as pd
import numpy as np
from datetime import timedelta, date, datetime

## my personal bank statments, exported as a csv and stored locally on my machine
spending_df = pd.read_csv('~/datasets/bank/spending_17May.CSV')#.set_index('Date')


In [70]:
""" Clean up df """

# remove transactions between my own bank accounts
spending_df = spending_df[spending_df['OP name'] != 'J D SHEA']
# remove deposits etc
spending_df = spending_df[spending_df['Amount'] < 0]

# make date in datetime format
spending_df['Date'] = pd.to_datetime(spending_df['Date'])

# flip amount col
spending_df['Cost'] = -spending_df['Amount']
# grab cols we care about
spending_df = spending_df[['Date', 'Memo/Description', 'Cost', 'Balance']]

# add date as integer col
spending_df['Date_int'] = spending_df['Date'].astype(np.int64)

                          

In [79]:
print(spending_df.size)
spending_df.sample(5)

1345


,Date,Memo/Description,Cost,Balance,Date_int
444,2020-05-13,POS W/D CRAIG'S CHEMI-12:28 ;,27.99,12.30,1589328000000000000
272,2019-12-13,POS W/D Seatoun Medic-10:00 ;,149.00,1.30,1576195200000000000
365,2020-02-24,POS W/D The Warehouse-13:50 ;,6.00,11.00,1582502400000000000
328,2020-01-02,FOUR SQUARE MARTINBORO MARTINBOROUGH ;,2.40,0.10,1577923200000000000
55,2019-01-07,POS W/D MOORE WILSONS-13:38 ;,7.00,0.13,1546819200000000000


In [80]:
print(f"Median cost: ${spending_df['Cost'].median()}")
print(f"Mean cost: ${spending_df['Cost'].mean()}")
print(f"Mode cost: ${spending_df['Cost'].mode()[0]}")

print("Some $10.00 transactions:")
spending_df[spending_df['Cost'] == 10.00].sample(10)

Median cost: $7.5
Mean cost: $14.319814126394053
Mode cost: $10.0
Some $10.00 transactions:


,Date,Memo/Description,Cost,Balance,Date_int
45,2019-06-16,WAREHOUSE MOBILE NEWMARKET ;,10.0,1.63,1560643200000000000
131,2019-08-08,TOMMY MILLIONS WELLINGTON ;,10.0,61.69,1565222400000000000
16,2019-05-30,TOMMY MILLIONS WELLINGTON ;,10.0,20.82,1559174400000000000
196,2019-10-22,POS W/D SNAPPER SERVI-20:48 ;,10.0,165.75,1571702400000000000
110,2019-07-30,POS W/D SNAPPER SERVI-17:15 ;,10.0,116.96,1564444800000000000
441,2020-10-05,TRADEME O361 PING Wellington ;,10.0,10.29,1601856000000000000
97,2019-07-17,WAREHOUSE MOBILE NEWMARKET ;,10.0,312.05,1563321600000000000
391,2020-11-03,VisaDebitAnnualFee *5206 ;,10.0,2.51,1604361600000000000
371,2020-02-26,POS W/D SNAPPER SERVI-15:39 ;,10.0,10.00,1582675200000000000
122,2019-02-08,TOMMY MILLIONS WELLINGTON ;,10.0,10.77,1549584000000000000


In [81]:
print(f"Average balance after purchase: ${spending_df['Balance'].mean()}")

Average balance after purchase: $51.706468401486994


In [83]:
print(f"Total spent: ${spending_df['Cost'].sum()}")

Total spent: $3852.03


In [97]:
print(f"Total of all sub-$50 costs: ${spending_df[spending_df['Cost'] < 50]['Cost'].sum()}")

Total of all sub-$50 costs: $2308.8999999999996
